# Batch Normalization

Teinar modelos profundos e fazê-los convergir numa quantidade razoável de tempo pode ser   uma tarefa complicada.
Nesta prática, descrevemos e usaremos o [*batch normalization*](https://arxiv.org/abs/1502.03167) (BN), uma técnica popular e eficaz capaz de acelerar a convergência de redes profundas que, juntamente com [blocos residuais](https://arxiv.org/abs/1512.03385), nos permitiu treinar redes com mais de 100 (até mesmo 1000) camadas.

Primeiro, vamos rever alguns dos desafios práticos ao treinar redes profundas.

1. O pré-processamento de dados geralmente se prova crucial modelagem estatística. Como falado anteriormente, num geral, se padroniza a entrada para ter uma média *zero* e variância de *um*. Padronizar os dados de entrada normalmente torna mais fácil treinar modelos profundos, pois os parâmetros estão, *a priori*, em uma escala similar.  
1. Para reles MLP ou CNN, enquanto treinamos o modelo, as ativações em camadas intermediárias da rede podem assumir diferentes ordens de magnitude. Os autores de [*batch normalization*](https://arxiv.org/abs/1502.03167) postulou que esta diferença na distribuição das ativações poderia dificultar a convergência da rede. Intuitivamente, poderíamos conjeturar que, se camada tem valores de ativação que são 100x que de outra camada, poderíamos precisa ajustar as taxas de aprendizagem de forma adaptável por camada (ou mesmo para neurônios dentro de uma mesma camada).
1. Redes mais profundas são complexas e propensas à *overfitting*. Isso significa que a regularização se torna mais importante. Empiricamente, notamos que mesmo com o *dropout*, os modelos podem cair numa situação de *overfitting*. Neste caso, devemos nos beneficiar de outras heurística de regularização.
 


In [1]:
!pip install mxnet-cu100

# imports basicos
# imports basicos
%matplotlib inline
import time, math, os, sys, numpy as np
from IPython import display
from matplotlib import pyplot as plt

import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, utils as gutils, data as gdata

# Tenta encontrar GPU
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()

In [0]:
# código para carregar o dataset do Fashion-MNIST
# https://github.com/zalandoresearch/fashion-mnist
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
    mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# funções básicas
def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n, l = nd.array([0]), 0, 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            # X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            y = y.astype('float32')
            y_hat = net(X)
            l += loss(y_hat, y).sum()
            acc_sum += (y_hat.argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n, l.asscalar() / n
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, ctx,
                   num_epochs):
    print('training on', ctx)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss, ctx)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## Detalhes Técnicos

Em 2015, uma heurística inteligente chamada [*batch normalization*](https://arxiv.org/abs/1502.03167) provou ser imensamente útil para melhorar a confiabilidade e a velocidade de convergência no treinamento de modelos profundos. Em cada iteração de treinamento, o  [*batch normalization*](https://arxiv.org/abs/1502.03167) normaliza as ativações de cada neurônio da camada oculta (em cada camada onde é aplicada) subtraindo sua média e dividindo pelo seu desvio padrão, estimando ambos com base no mini-batch atual. Note que se o tamanho do batch fosse $1$, não aprenderíamos nada porque durante o treinamento, todos os nó levaria valor $0$. No entanto, com minibatches grandes o suficiente, a abordagem se prova eficaz e estável.

Em poucas palavras, a ideia do [*batch normalization*](https://arxiv.org/abs/1502.03167) é transformar a ativação em uma determinada camada de $\mathbf{x}$ para:

$$\mathrm{BN}(\mathbf{x}) = \mathbf{\gamma} \odot \frac{\mathbf{x} - \hat{\mathbf{\mu}}}{\hat\sigma} + \mathbf{\beta}$$

Aqui, $\hat{\mathbf{\mu}}$ é a estimativa da média e $\hat {\mathbf{\sigma}}$ é a estimativa da variância. O resultado é que as ativações são aproximadamente reescaladas para uma média zero e uma variância unitária. Como podemos notar, as ativações das camadas intermediárias não pode divergir muito pois estamos ativamente redimensionando-as de volta para uma dada ordem de grandeza através de $\mathbf{\mu}$ e $\sigma$. Entretanto, em alguns casos, as ativações pode precisar diferir dos dados padronizados. Para lidar com isso e dar mais liberdade à essa normalização, definimos um coeficiente de escala de coordenadas $\mathbf{\gamma}$ e um offset $\mathbf{\beta}$. Intuitivamente, espera-se que essa normalização nos permita ser mais agressivo ao escolher taxas de aprendizado maiores.

Em princípio, podemos querer usar todos os nossos dados de treinamento para estimar a média e variância. No entanto, as ativações correspondentes a cada exemplo mudar cada vez que atualizamos nosso modelo. Para remediar este problema, o [*batch normalization*](https://arxiv.org/abs/1502.03167) usa apenas o minibatch atual para estimar $\hat{\mathbf {\mu}} $ e $\hat \sigma$. É justamente por esse fato de normalizamos com base apenas no *batch* atual que  o método se chama *batch normalization*. Para indicar qual *minibatch* $\mathcal {B}$ é usado, nós denotamos essas variáveis como $\hat{\mathbf{\mu}}_\mathcal {B}$ e $\hat \sigma_\mathcal {B}$.

$$\hat{\mathbf{\mu}}_\mathcal{B} \leftarrow \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x}$$

$$\hat{\mathbf{\sigma}}_\mathcal{B}^2 \leftarrow \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \mathbf{\mu}_{\mathcal{B}})^2 + \epsilon$$

Observe que adicionamos uma pequena constante $\epsilon > 0$ à estimativa de variância para garantir que nunca acabemos por dividir por zero, mesmo nos casos em que a estimativa de variância pode desaparecer por acidente.
Vamos agora ver, na prática, o uso do [*batch normalization*](https://arxiv.org/abs/1502.03167).

## Camadas de *Batch Normalization*

O [*batch normalization*](https://arxiv.org/abs/1502.03167) para camadas totalmente conectadas (*fully-connected* ou Densas) e camadas convolucionais são ligeiramente diferentes. Isso se deve à dimensionalidade dos dados gerados pelas camadas convolucionais. Os dois casos são discutidos abaixo. Observe que uma das principais diferenças entre a camada de [*batch normalization*](https://arxiv.org/abs/1502.03167) e outras camadas é que a primeira opera em um *minibatch* completo de cada vez (caso contrário, não é possível calcular os parâmetros de média e variância por *batch*).

### Camadas densas

Normalmente, aplicamos a camada de [*batch normalization*](https://arxiv.org/abs/1502.03167) entre a transformação e a função de ativação em uma camada densa. A seguir, denotamos por $\mathbf{u}$ a entrada e por $\mathbf{x} = \mathbf{W}\mathbf{u} + \mathbf{b}$ a saída da transformada linear. Isso produz a seguinte fórmula:

$$\mathbf{y} = \phi(\mathrm{BN}(\mathbf{x})) =  \phi(\mathrm{BN}(\mathbf{W}\mathbf{u} + \mathbf{b}))$$

Lembre-se de que a média e a variância são calculadas no **mesmo** *minibatch* $\mathcal{B}$ no qual a transformação é aplicada. Lembre-se também que o coeficiente $\mathbf{\gamma}$ e o offset $\mathbf{\beta}$ são parâmetros que precisam ser aprendidos. Eles garantem que o efeito do [*batch normalization*](https://arxiv.org/abs/1502.03167) possa ser neutralizado conforme necessário.

### Camadas convolucionais

Para camadas convolucionais, o [*batch normalization*](https://arxiv.org/abs/1502.03167) ocorre após o cálculo da convolução e antes da aplicação da função de ativação. Se a computação de convolução gerar múltiplos canais, realizamos o [*batch normalization*](https://arxiv.org/abs/1502.03167) para **cada** uma das saídas desses canais, que tem parâmetros ($\mathbf{\gamma}$ e $\mathbf{\beta}$) independentes. Suponha que haja exemplos de $m$ no *batch*. Em um único canal, assumimos que a altura e a largura da saída da convolução são $p$ e $q$, respectivamente. Precisamos realizar o [*batch normalization*](https://arxiv.org/abs/1502.03167) para $m \times p \times q$ elementos neste canal simultaneamente. Ao executar o cálculo de padronização para esses elementos, usamos a mesma média e variância. Em outras palavras, usamos as médias e as variâncias dos elementos $m \times p \times q$ neste canal em vez de um por pixel.


## MXNet e o caso de estudo LeNet-5

Agora vamos implementar a [LeNet-5](https://ieeexplore.ieee.org/document/726791) usando a camada de [*batch normalization*](https://arxiv.org/abs/1502.03167).

Em frameworks modernos, camadas de [*batch normalization*](https://mxnet.incubator.apache.org/api/python/gluon/nn.html#mxnet.gluon.nn.BatchNorm) já vem implementadas e são fáceis de usar.

In [15]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 10, 0.01, 128, 0.0001

# rede baseada na LeNet-5 
net = nn.Sequential()
net.add(nn.Conv2D(6, kernel_size=5, strides=1, padding=0),   # entrada: (b, 1, 32, 32) e saida: (b, 6, 28, 28)
        nn.BatchNorm(),
        nn.Activation('tanh'),
        nn.AvgPool2D(pool_size=2, strides=2, padding=0),     # entrada: (b, 6, 28, 28) e saida: (b, 6, 14, 14)
        nn.Conv2D(16, kernel_size=5, strides=1, padding=0),  # entrada: (b, 6, 14, 14) e saida: (b, 16, 10, 10)
        nn.BatchNorm(),
        nn.Activation('tanh'),
        nn.AvgPool2D(pool_size=2, strides=2, padding=0),     # entrada: (b, 16, 10, 10) e saida: (b, 16, 5, 5)
        nn.Conv2D(120, kernel_size=5, strides=1, padding=0), # entrada: (b, 16, 5, 5) e saida: (b, 120, 1, 1)
        nn.BatchNorm(),
        nn.Activation('tanh'),
        nn.Flatten(),  # lineariza formando um vetor         # entrada: (b, 120, 1, 1) e saida: (b, 120*1*1) = (b, 120)
        nn.Dense(84),
        nn.BatchNorm(),
        nn.Activation('tanh'),
        nn.Dense(10))
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=32)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr, 'wd': wd_lambda})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 0.5455, train acc 0.797, test loss 0.4089, test acc 0.851, time 10.3 sec
epoch 2, train loss 0.3727, train acc 0.863, test loss 0.3609, test acc 0.866, time 10.1 sec
epoch 3, train loss 0.3327, train acc 0.878, test loss 0.3802, test acc 0.854, time 10.0 sec
epoch 4, train loss 0.3106, train acc 0.887, test loss 0.3783, test acc 0.863, time 10.0 sec
epoch 5, train loss 0.2926, train acc 0.894, test loss 0.2965, test acc 0.891, time 10.0 sec
epoch 6, train loss 0.2887, train acc 0.895, test loss 0.3855, test acc 0.860, time 10.2 sec
epoch 7, train loss 0.2805, train acc 0.897, test loss 0.3206, test acc 0.885, time 10.0 sec
epoch 8, train loss 0.2704, train acc 0.901, test loss 0.3617, test acc 0.859, time 12.3 sec
epoch 9, train loss 0.2720, train acc 0.901, test loss 0.4826, test acc 0.820, time 13.3 sec
epoch 10, train loss 0.2643, train acc 0.904, test loss 0.2854, test acc 0.897, time 10.8 sec
